In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ph-final-ls-ds/ph_final.csv
/kaggle/input/phdata/Final_merged_ph_landsat.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from math import sqrt
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold,RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import StandardScaler  
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
import joblib

In [3]:
df=pd.read_csv('/kaggle/input/phdata/Final_merged_ph_landsat.csv')
df.head()

,SiteID,Date,Band1_Mean,Band2_Mean,Band3_Mean,Band4_Mean,Band5_Mean,PH
0,BLUE_CHALK,2015-06-03 00:00:00,0.125024,0.101374,0.074620,0.053516,0.086735,6.77
1,BLUE_CHALK,2015-07-15 00:00:00,0.178984,0.150875,0.111050,0.084422,0.131796,6.60
2,BLUE_CHALK,2015-08-28 00:00:00,0.200051,0.168958,0.120381,0.087537,0.130874,6.85
3,BLUE_CHALK,2015-11-05 00:00:00,0.127767,0.100123,0.060282,0.040461,0.036792,6.66
4,BLUE_CHALK,2015-11-17 00:00:00,0.214717,0.176831,0.115975,0.093581,0.091780,6.69


In [4]:
df.dropna(inplace=True)
print(df.count())

SiteID        593
Date          593
Band1_Mean    593
Band2_Mean    593
Band3_Mean    593
Band4_Mean    593
Band5_Mean    593
PH            593
dtype: int64


In [5]:
for i in range(1, 6):
    for j in range(1, 6):
        if i!=j:
            new_column_name = f'B{i}_B{j}_Ratio'
            print(new_column_name)
            df[new_column_name] = df[f'Band{i}_Mean'] / df[f'Band{j}_Mean']
# df.head()

B1_B2_Ratio
B1_B3_Ratio
B1_B4_Ratio
B1_B5_Ratio
B2_B1_Ratio
B2_B3_Ratio
B2_B4_Ratio
B2_B5_Ratio
B3_B1_Ratio
B3_B2_Ratio
B3_B4_Ratio
B3_B5_Ratio
B4_B1_Ratio
B4_B2_Ratio
B4_B3_Ratio
B4_B5_Ratio
B5_B1_Ratio
B5_B2_Ratio
B5_B3_Ratio
B5_B4_Ratio


> **Support Vector Regression**

In [6]:
features = [
    'Band1_Mean', 'Band2_Mean', 'Band3_Mean', 'Band4_Mean', 'Band5_Mean',
    'B1_B2_Ratio','B1_B3_Ratio','B1_B4_Ratio','B1_B5_Ratio','B2_B1_Ratio','B2_B3_Ratio',
    'B2_B4_Ratio','B2_B5_Ratio','B3_B1_Ratio','B3_B2_Ratio','B3_B4_Ratio','B3_B5_Ratio','B4_B1_Ratio',
    'B4_B2_Ratio','B4_B3_Ratio','B4_B5_Ratio','B5_B1_Ratio','B5_B2_Ratio','B5_B3_Ratio','B5_B4_Ratio'
]

label = ['PH']

X = df.loc[:, features].values
y = df.loc[:, label].values

#Test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Model Development
svr_model = SVR(kernel='rbf')

param_dist = {
    'C': [x for x in range(1, 10001, 1)],
    'epsilon': [x for x in np.arange(0.001, 1001, 0.001)]
}

random_search = RandomizedSearchCV(SVR(), param_distributions=param_dist, n_iter=200, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

randomDf = pd.DataFrame(random_search.cv_results_)
best_svr_model = random_search.best_estimator_

y_pred = best_svr_model.predict(X_test)

#Model Evaluation
test_r2 = r2_score(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Best Parameters: {random_search.best_params_}")
print(f"R2 Score: {test_r2}")
print(f"RMSE on Test Set: {test_rmse}")

svr_model_filename = 'ph-svr.joblib'
joblib.dump(best_svr_model, svr_model_filename)
print(f'Best SVR model saved as {svr_model_filename}')

Best Parameters: {'epsilon': 0.182, 'C': 4322}
R2 Score: 0.5121888780685605
RMSE on Test Set: 0.9078357632009152
Best SVR model saved as ph-svr.joblib


> **Random Forest**

In [7]:
from sklearn.ensemble import RandomForestRegressor

features = [
    'Band1_Mean', 'Band2_Mean', 'Band3_Mean', 'Band4_Mean', 'Band5_Mean',
    'B1_B2_Ratio','B1_B3_Ratio','B1_B4_Ratio','B1_B5_Ratio','B2_B1_Ratio','B2_B3_Ratio',
    'B2_B4_Ratio','B2_B5_Ratio','B3_B1_Ratio','B3_B2_Ratio','B3_B4_Ratio','B3_B5_Ratio','B4_B1_Ratio',
    'B4_B2_Ratio','B4_B3_Ratio','B4_B5_Ratio','B5_B1_Ratio','B5_B2_Ratio','B5_B3_Ratio','B5_B4_Ratio'
]

label = ['PH']

X = df.loc[:, features].values
y = df.loc[:, label].values

#Test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Model Development
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_
y_pred_tuned = best_rf_model.predict(X_test)

#Model Evaluate
r2_tuned = r2_score(y_test, y_pred_tuned)
rmse_rf= np.sqrt(mean_squared_error(y_test, y_pred))

print(f'Best Hyperparameters: {best_params}\n')

print(f'R2 Score: {r2_tuned}')
print(f"RMSE: {rmse_rf}")

rf_model_filename = 'ph-rf.joblib'
joblib.dump(best_rf_model, rf_model_filename)
print(f'Best-tuned RandomForestRegressor model saved as {rf_model_filename}')

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

R2 Score: 0.4686702138237103
RMSE: 0.9078357632009152
Best-tuned RandomForestRegressor model saved as ph-rf.joblib


> **CatBoost Regression**

In [8]:
from catboost import CatBoostRegressor

features = [
    'Band1_Mean', 'Band2_Mean', 'Band3_Mean', 'Band4_Mean', 'Band5_Mean',
    'B1_B2_Ratio','B1_B3_Ratio','B1_B4_Ratio','B1_B5_Ratio','B2_B1_Ratio',
    'B2_B3_Ratio','B2_B4_Ratio','B2_B5_Ratio','B3_B1_Ratio','B3_B2_Ratio',
    'B3_B4_Ratio','B3_B5_Ratio','B4_B1_Ratio','B4_B2_Ratio','B4_B3_Ratio',
    'B4_B5_Ratio','B5_B1_Ratio','B5_B2_Ratio','B5_B3_Ratio','B5_B4_Ratio'
]

label = ['PH']

X = df.loc[:, features].values
y = df.loc[:, label].values

#Test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Model Development
catboost_model = CatBoostRegressor(iterations=500, depth=15, learning_rate=1, loss_function='RMSE', random_seed=42)
catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)
y_pred = catboost_model.predict(X_test)

# Evaluate the model
print()
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score on Test Set: {r2}")
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse}')

catboost_model_filename = 'ph-catboost.joblib'
joblib.dump(catboost_model, catboost_model_filename)
print(f'CatBoostRegressor model saved as {catboost_model_filename}')

0:	learn: 0.8997195	test: 1.0877652	best: 1.0877652 (0)	total: 2.5s	remaining: 20m 49s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 1.048462207
bestIteration = 15

Shrink model to first 16 iterations.

R^2 Score on Test Set: 0.3493571122624216
RMSE: 1.0484622102122705
CatBoostRegressor model saved as ph-catboost.joblib


> **Feed Forward Neural Network**

In [9]:
from keras import models, layers, optimizers, regularizers
from sklearn import model_selection, preprocessing
import tensorflow as tf
from tqdm import tqdm
from scipy import stats
from tensorflow.keras.callbacks import EarlyStopping

2024-03-18 01:23:20.441715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 01:23:20.441843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 01:23:20.585386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
features = [
    'Band1_Mean', 'Band2_Mean', 'Band3_Mean', 'Band4_Mean', 'Band5_Mean',
    'B1_B2_Ratio','B1_B3_Ratio','B1_B4_Ratio','B1_B5_Ratio','B2_B1_Ratio',
    'B2_B3_Ratio','B2_B4_Ratio','B2_B5_Ratio','B3_B1_Ratio','B3_B2_Ratio',
    'B3_B4_Ratio','B3_B5_Ratio','B4_B1_Ratio','B4_B2_Ratio','B4_B3_Ratio',
    'B4_B5_Ratio','B5_B1_Ratio','B5_B2_Ratio','B5_B3_Ratio','B5_B4_Ratio'
]

label = ['PH']

X = df.loc[:, features].values
y = df.loc[:, label].values

#Test-train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Model Development - sequential model
model = models.Sequential()
model.add(layers.Dense(units=32, input_dim=25, activation='relu'))
model.add(layers.Dense(units=64,input_dim=32,activation='relu'))
model.add(layers.Dense(units=128,input_dim=64, activation='relu'))
model.add(layers.Dense(units=256,input_dim=128, activation='relu'))
# model.add(layers.Dense(units=512,input_dim=256, activation='relu'))
model.add(layers.Dense(units=256,input_dim=512, activation='relu'))
model.add(layers.Dense(units=128,input_dim=256, activation='relu'))
model.add(layers.Dense(units=64,input_dim=128, activation='relu'))
model.add(layers.Dense(units=1, activation='linear'))

model.compile(loss='mean_squared_error',optimizer=optimizers.Adam(lr=0.0001))
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history=model.fit(X_train, y_train, epochs=100, batch_size=2, callbacks=[early_stopping])

Epoch 1/100


I0000 00:00:1710725045.532471     123 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


237/237 [==============================] - 5s 3ms/step - loss: 4.1801
Epoch 2/100
237/237 [==============================] - 1s 3ms/step - loss: 1.9760
Epoch 3/100
237/237 [==============================] - 1s 3ms/step - loss: 1.9599
Epoch 4/100
237/237 [==============================] - 1s 3ms/step - loss: 1.6928
Epoch 5/100
237/237 [==============================] - 1s 3ms/step - loss: 1.5529
Epoch 6/100
237/237 [==============================] - 1s 3ms/step - loss: 1.6124
Epoch 7/100
237/237 [==============================] - 1s 3ms/step - loss: 1.6095
Epoch 8/100
237/237 [==============================] - 1s 3ms/step - loss: 1.4606
Epoch 9/100
237/237 [==============================] - 1s 3ms/step - loss: 1.4363
Epoch 10/100
237/237 [==============================] - 1s 3ms/step - loss: 1.3481
Epoch 11/100
237/237 [==============================] - 1s 3ms/step - loss: 1.3391
Epoch 12/100
237/237 [==============================] - 1s 3ms/step - loss: 1.3547
Epoch 13/100
237/237 [===

In [ ]:
predictions = model.predict(X_test)

#Model Evaluation

from sklearn import metrics

print("R2: ",metrics.explained_variance_score(y_test,predictions))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test,predictions)))

In [ ]:
model.save("ph-feednn.h5")
import pickle

# Assuming 'data' is your Python object (e.g., a dictionary)
with open('ph-data.pkl', 'wb') as f:
    pickle.dump(df, f)
